# Bundesdatascrap


Getting NRW's COVID data from : https://www.service.bund.de/Content/DE/Ausschreibungen/Suche/Formular.html?nn=4641482&cl2Addresses_Adresse_State=nordrhein-westfalen&resultsPerPage=100
Based on the tutorials: 
- https://medium.com/opex-analytics/simple-web-scraping-in-python-90d6fddfaeca
- https://medium.com/analytics-vidhya/beginner-to-advance-web-scraping-guide-in-python-799ffd367067
- https://stackoverflow.com/questions/48513576/python-requests-module-doesnt-return-full-page-during-get-request
- https://medium.com/opex-analytics/simple-web-scraping-in-python-90d6fddfaeca


In [1]:
import requests
import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

In [2]:
# Finding URL pattern:

#first 100 cases , it has 23 more
url0 = 'https://www.service.bund.de/Content/DE/Ausschreibungen/Suche/Formular.html?nn=4641482&cl2Addresses_Adresse_State=nordrhein-westfalen&resultsPerPage=100'
url1 = 'https://www.service.bund.de/Content/DE/Ausschreibungen/Suche/Formular.html?nn=4641482&gtp=4642258_list%253D2&type=0&searchResult=true&cl2Addresses_Adresse_State=nordrhein-westfalen&resultsPerPage=100'
url2 = 'https://www.service.bund.de/Content/DE/Ausschreibungen/Suche/Formular.html?nn=4641482&gtp=4642258_list%253D3&type=0&searchResult=true&cl2Addresses_Adresse_State=nordrhein-westfalen&resultsPerPage=100'
url3 = 'https://www.service.bund.de/Content/DE/Ausschreibungen/Suche/Formular.html?nn=4641482&gtp=4642258_list%253D4&type=0&searchResult=true&cl2Addresses_Adresse_State=nordrhein-westfalen&resultsPerPage=100'

# Found, creating a base url for pages 1:23 , page 0 must be url0

# creating list of all urls:

results = [url0, url1]

for i in range(2,24):
    i = str(i)    # converting to str
    url_base = 'https://www.service.bund.de/Content/DE/Ausschreibungen/Suche/Formular.html?nn=4641482&gtp=4642258_list%253D' + i + '&type=0&searchResult=true&cl2Addresses_Adresse_State=nordrhein-westfalen&resultsPerPage=100'
    results.append(url_base)
    
#results

# Scrap

Here we begging with scrap. <br>
I saw on mozila firefox that the part of the page that I wanted to scrap what is inside the "ul class "result-list""

![title](html_bundesdatei2.png)

In [3]:
# Scrap

# makes a request to the web page and gets its HTML
r = requests.get(url0,headers={'User-Agent': 'Mozilla/5.0'}) # with the user agent it downloaded all the page

# stores the HTML page in 'soup', a BeautifulSoup object
soup = BeautifulSoup(r.text, 'html.parser')

In [122]:
#looking at what I get inside the ul markers
# print(soup.find_all('ul'))

bs4.element.ResultSet

In [4]:
#Creating an object that will contain all the results inside the marker ul class:"result-list"
content = soup.find('ul', {"class": "result-list"})

content #Uhul! 

<ul class="result-list">
<li>
<a href="IMPORTE/Ausschreibungen/subreport/2021/03/E34257533.html;jsessionid=5778B38234B98C7BA0803CB6CA0599C5.1_cid296?nn=4641482&amp;type=0&amp;searchResult=true" title="Zur Ausschreibung 'Sanierung Heidberghalle-Metallbau-und Verglasungsarbeiten'"><div aria-labelledby="title">
<h3><em>Ausschreibung</em>Sa­nie­rung Heid­berg­hal­le-Me­tall­bau-und Ver­gla­sungs­ar­bei­ten</h3>
<p><em>Vergabestelle</em> Marktstadt Waldbröl - Der Bürgermeister</p>
</div>
<div aria-labelledby="date">
<p><em>Veröffentlicht</em> 04.03.21</p>
</div>
<div aria-labelledby="location">
<p><em>Angebotsfrist</em> 07.04.21</p>
</div>
</a>
</li>
<li>
<a href="IMPORTE/Ausschreibungen/subreport/2021/03/E24882845.html;jsessionid=5778B38234B98C7BA0803CB6CA0599C5.1_cid296?nn=4641482&amp;type=0&amp;searchResult=true" title="Zur Ausschreibung 'Sanierung Heidberghalle: Sportboden,Prallschutz'"><div aria-labelledby="title">
<h3><em>Ausschreibung</em>Sa­nie­rung Heid­berg­hal­le: Sport­bo­den,Pr

More specific, I want the following structure to build my dataframe:
    
![title](html_bundesdatei3.png)

In [5]:
# what I want is insite the marker a, so Im going to get the content and get only the a inside it:
links = content.find_all('a')#[1].get_text()
links # each a marker is one row

[<a href="IMPORTE/Ausschreibungen/subreport/2021/03/E34257533.html;jsessionid=5778B38234B98C7BA0803CB6CA0599C5.1_cid296?nn=4641482&amp;type=0&amp;searchResult=true" title="Zur Ausschreibung 'Sanierung Heidberghalle-Metallbau-und Verglasungsarbeiten'"><div aria-labelledby="title">
 <h3><em>Ausschreibung</em>Sa­nie­rung Heid­berg­hal­le-Me­tall­bau-und Ver­gla­sungs­ar­bei­ten</h3>
 <p><em>Vergabestelle</em> Marktstadt Waldbröl - Der Bürgermeister</p>
 </div>
 <div aria-labelledby="date">
 <p><em>Veröffentlicht</em> 04.03.21</p>
 </div>
 <div aria-labelledby="location">
 <p><em>Angebotsfrist</em> 07.04.21</p>
 </div>
 </a>,
 <a href="IMPORTE/Ausschreibungen/subreport/2021/03/E24882845.html;jsessionid=5778B38234B98C7BA0803CB6CA0599C5.1_cid296?nn=4641482&amp;type=0&amp;searchResult=true" title="Zur Ausschreibung 'Sanierung Heidberghalle: Sportboden,Prallschutz'"><div aria-labelledby="title">
 <h3><em>Ausschreibung</em>Sa­nie­rung Heid­berg­hal­le: Sport­bo­den,Prall­schutz</h3>
 <p><em>Ver

In [6]:
# Teste
df = pd.DataFrame(links)
df = df.iloc[:, [0,2,4]]
df = df.rename(columns={0: "ausschreibung",
                        2: "veroeffentlicht",
                        4:"angebotsfrist"})

/Users/user/opt/anaconda3/lib/python3.8/site-packages/pandas/core/internals/construction.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


In [23]:
#Não está dando certo:
#df = df.replace({', \n\]': ''}, regex=True)
#df

,ausschreibung,veroeffentlicht,angebotsfrist
0,"[\n, [[Ausschreibung], Sa­nie­rung Heid­berg­h...","[\n, [[Veröffentlicht], 04.03.21], \n]","[\n, [[Angebotsfrist], 07.04.21], \n]"
1,"[\n, [[Ausschreibung], Sa­nie­rung Heid­berg­h...","[\n, [[Veröffentlicht], 04.03.21], \n]","[\n, [[Angebotsfrist], 07.04.21], \n]"
2,"[\n, [[Ausschreibung], Neu­in­stal­la­ti­on de...","[\n, [[Veröffentlicht], 04.03.21], \n]","[\n, [[Angebotsfrist], 06.04.21], \n]"
3,"[\n, [[Ausschreibung], Sink­kas­ten­rei­ni­gun...","[\n, [[Veröffentlicht], 04.03.21], \n]","[\n, [[Angebotsfrist], 01.04.21], \n]"
4,"[\n, [[Ausschreibung], Leicht­ath­le­tik­an­la...","[\n, [[Veröffentlicht], 04.03.21], \n]","[\n, [[Angebotsfrist], 24.03.21], \n]"
...,...,...,...
95,"[\n, [[Ausschreibung], Aus­tausch BHKW An­la­g...","[\n, [[Veröffentlicht], 03.03.21], \n]","[\n, [[Angebotsfrist], 25.03.21], \n]"
96,"[\n, [[Ausschreibung], Neu­bau Mehr­fa­mi­li­e...","[\n, [[Veröffentlicht], 03.03.21], \n]","[\n, [[Angebotsfrist], 23.03.21], \n]"
97,"[\n, [[Ausschreibung], Au­ßen­an­la­gen - Ki­t...","[\n, [[Veröffentlicht], 03.03.21], \n]","[\n, [[Angebotsfrist], 25.03.21], \n]"
98,"[\n, [[Ausschreibung], Tisch­ler­ar­bei­ten - ...","[\n, [[Veröffentlicht], 03.03.21], \n]","[\n, [[Angebotsfrist], 22.03.21], \n]"
